In [1]:
# 問題1 TensorFLow 公式チュートリアル、クイックスタート
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
# Dropoutは過学習抑制
#  dense 出力ユニット数
# 28 * 28 -> 128 -> 10とoutputしてく
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

2022-07-14 19:31:03.405142: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
x_train[:1].shape

(1, 28, 28)

In [5]:
# fitはしてないので、初期値の状態でpredict
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.63307196,  0.05814474, -0.4438566 , -0.3242117 , -0.3630253 ,
        -0.15097958, -0.7268583 , -0.02101578, -0.47125277,  0.06124397]],
      dtype=float32)

In [6]:
# 確率に変換
tf.nn.softmax(predictions).numpy()

array([[0.06930735, 0.13834736, 0.08374415, 0.09438774, 0.09079439,
        0.11224043, 0.06310276, 0.12781796, 0.08148102, 0.1387768 ]],
      dtype=float32)

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# 損失値出力
loss_fn(y_train[:1], predictions).numpy()

2.1871119

In [8]:
# compile
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [9]:
# 学習
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2961 - accuracy: 0.9144
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1402 - accuracy: 0.9581
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1046 - accuracy: 0.9687
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0846 - accuracy: 0.9742
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0715 - accuracy: 0.9772


In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0713 - accuracy: 0.9776 - 335ms/epoch - 1ms/step


[0.07134812325239182, 0.9775999784469604]

In [11]:
# modelの構成を変えたときの損失値を正確に検証するためにsoftmaxを最後に組むのは推奨しないが、softmaxがないと検証しにくいので今回は追加
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.8936304e-08, 5.8649015e-09, 2.0991276e-07, 2.6158363e-04,
        1.5539249e-10, 9.2479858e-08, 1.4027981e-12, 9.9972612e-01,
        8.1938202e-07, 1.1064676e-05],
       [2.2185669e-07, 6.0557610e-05, 9.9964774e-01, 2.8936908e-04,
        8.2309744e-16, 5.3097511e-07, 7.7565268e-08, 9.5041639e-16,
        1.4266934e-06, 5.2102241e-13],
       [6.8489143e-08, 9.9833727e-01, 1.8002678e-04, 9.3919349e-05,
        1.1778935e-04, 1.2318669e-05, 9.2140479e-05, 4.9781136e-04,
        6.6753762e-04, 1.0536229e-06],
       [9.9991810e-01, 5.0968751e-08, 7.7546720e-05, 8.0600799e-08,
        1.6959751e-07, 1.3666025e-07, 2.7702749e-06, 4.1045197e-07,
        4.1297548e-09, 7.0309352e-07],
       [6.2672792e-05, 6.5024630e-10, 9.3228942e-07, 1.9634840e-07,
        9.9221742e-01, 2.8050945e-06, 1.2174367e-05, 2.0280630e-04,
        2.7318431e-06, 7.4982652e-03]], dtype=float32)>

In [13]:
### functional APIのチュートリアル
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print("x_train: ", x_train.shape)
print("x_test: ", x_test.shape)

x_train:  (60000, 28, 28, 1)
x_test:  (10000, 28, 28, 1)


In [14]:
# 10000件ずつの範囲でshuffleして32件ずつ抽出できるようにしている
# tf.data.Datasetにはdata pipelineを組むためにこのような機能を用意している
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
print("train_ds: ", train_ds)
print("test_ds: ", test_ds)

train_ds:  <BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>
test_ds:  <BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [15]:
# model class定義
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [16]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [17]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [18]:
# tf.GradientTapeでモデルを学習
# tf.functionにより見通しが良くなる
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [19]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [20]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

        template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.03378335013985634, Test Accuracy: 100.0
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.027627594769001007, Test Accuracy: 100.0
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.021066943183541298, Test Accuracy: 100.0
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.04190182313323021, Test Accuracy: 99.21875
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.039013177156448364, Test Accuracy: 99.375
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.03406136855483055, Test Accuracy: 99.47917175292969
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.038027551025152206, Test Accuracy: 99.10713958740234
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.046569984406232834, Test Accuracy: 98.828125
Epoch 1, Loss: 0.1366226077

Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.0881325975060463, Test Accuracy: 97.05105590820312
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.09020456671714783, Test Accuracy: 97.0486068725586
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08978188037872314, Test Accuracy: 97.04623413085938
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08915809541940689, Test Accuracy: 97.0439224243164
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.09069301933050156, Test Accuracy: 96.95833587646484
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.09115733951330185, Test Accuracy: 96.95723724365234
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.09263775497674942, Test Accuracy: 96.91558837890625
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.0922546535730362, Test 

Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08766111731529236, Test Accuracy: 97.29610443115234
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08745139837265015, Test Accuracy: 97.29312896728516
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08750203251838684, Test Accuracy: 97.26835632324219
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08778714388608932, Test Accuracy: 97.265625
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08804024010896683, Test Accuracy: 97.26293182373047
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.0878174677491188, Test Accuracy: 97.26026916503906
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08744130283594131, Test Accuracy: 97.2789077758789
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.08702611923217773, Test Accura

Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.07772701233625412, Test Accuracy: 97.51184844970703
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.07777039706707001, Test Accuracy: 97.49410247802734
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.07744193077087402, Test Accuracy: 97.50586700439453
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.07717027515172958, Test Accuracy: 97.51752471923828
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.07682294398546219, Test Accuracy: 97.52906799316406
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.07648526877164841, Test Accuracy: 97.54051208496094
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.0761488527059555, Test Accuracy: 97.55184173583984
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.07582395523786545, Te

Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.06256116926670074, Test Accuracy: 98.00836181640625
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.062347412109375, Test Accuracy: 98.01535034179688
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.062137629836797714, Test Accuracy: 98.02228546142578
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.061923813074827194, Test Accuracy: 98.0291748046875
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.06173396110534668, Test Accuracy: 98.03602600097656
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.06152593344449997, Test Accuracy: 98.0428237915039
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.06132759898900986, Test Accuracy: 98.04956817626953
Epoch 1, Loss: 0.1366226077079773, Accuracy: 95.92000579833984, Test Loss: 0.06126532331109047, Tes

...

Epoch 4, Loss: 0.013224240392446518, Accuracy: 99.55500030517578, Test Loss: 0.06538240611553192, Test Accuracy: 98.18096923828125
Epoch 4, Loss: 0.013224240392446518, Accuracy: 99.55500030517578, Test Loss: 0.06514085829257965, Test Accuracy: 98.18772888183594
Epoch 4, Loss: 0.013224240392446518, Accuracy: 99.55500030517578, Test Loss: 0.06490068882703781, Test Accuracy: 98.19444274902344
Epoch 4, Loss: 0.013224240392446518, Accuracy: 99.55500030517578, Test Loss: 0.06466124951839447, Test Accuracy: 98.20111083984375
Epoch 4, Loss: 0.013224240392446518, Accuracy: 99.55500030517578, Test Loss: 0.06442367285490036, Test Accuracy: 98.20771789550781
Epoch 4, Loss: 0.013224240392446518, Accuracy: 99.55500030517578, Test Loss: 0.06418778747320175, Test Accuracy: 98.21428680419922
Epoch 4, Loss: 0.013224240392446518, Accuracy: 99.55500030517578, Test Loss: 0.06395357847213745, Test Accuracy: 98.2208023071289
Epoch 4, Loss: 0.013224240392446518, Accuracy: 99.55500030517578, Test Loss: 0.06372

Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07574915885925293, Test Accuracy: 98.4375
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07594487071037292, Test Accuracy: 98.39285278320312
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07384112477302551, Test Accuracy: 98.4375
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.08504737168550491, Test Accuracy: 98.3108139038086
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.08315855264663696, Test Accuracy: 98.35526275634766
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.10200563818216324, Test Accuracy: 98.15705108642578
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.10167306661605835, Test Accuracy: 98.125
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.09928326308727264, Test Accuracy: 98

Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.09673187136650085, Test Accuracy: 97.83878326416016
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.09600494801998138, Test Accuracy: 97.85879516601562
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.09515330940485, Test Accuracy: 97.8784408569336
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.09546636790037155, Test Accuracy: 97.86931610107422
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.09808748960494995, Test Accuracy: 97.86035919189453
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.10021273791790009, Test Accuracy: 97.82366180419922
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.0993412509560585, Test Accuracy: 97.8429183959961
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.0984810441

Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.08898169547319412, Test Accuracy: 97.9225845336914
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.08892003446817398, Test Accuracy: 97.91667175292969
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.08842631429433823, Test Accuracy: 97.9283676147461
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.0879327654838562, Test Accuracy: 97.93994140625
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.08851802349090576, Test Accuracy: 97.93402862548828
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.08802909404039383, Test Accuracy: 97.94544219970703
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.08754564821720123, Test Accuracy: 97.95672607421875
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.0870679020

Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07191796600818634, Test Accuracy: 98.25697326660156
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07176309078931808, Test Accuracy: 98.2514877319336
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07169201225042343, Test Accuracy: 98.24604797363281
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07141321152448654, Test Accuracy: 98.2529525756836
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07117761671543121, Test Accuracy: 98.25980377197266
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07090071588754654, Test Accuracy: 98.2666015625
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.07062505930662155, Test Accuracy: 98.27334594726562
Epoch 5, Loss: 0.008975173346698284, Accuracy: 99.70999908447266, Test Loss: 0.0703996568